In [1]:
from sklearn.decomposition import PCA #PCA算法
from sklearn.ensemble import RandomForestClassifier as RFC #随机森林分类模型
from sklearn.model_selection import cross_val_score #交叉验证
from sklearn.neighbors import KNeighborsClassifier as KNN #KNN算法
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorcircuit as tc
from functools import partial
from sklearn.model_selection import train_test_split
K = tc.set_backend("tensorflow")

In [24]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc
K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_3_or_6 = (y_train1 == 3) | (y_train1 == 6)  
filter_3_or_6_test = (y_test1 == 3) | (y_test1 == 6)

x_train = x_train1[filter_3_or_6 ]  
y_train = y_train1[filter_3_or_6 ]  
  
x_test = x_test1[filter_3_or_6_test]  
y_test = y_test1[filter_3_or_6_test]  

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 0
    else:
        y_train[i] = 1
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 0
    else:
        y_test[i] = 1

  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)


batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)

y_train = tf.one_hot(y_train,depth=2,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=2,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=2

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
189/189 [==============================] - 255s 782ms/step - loss: 0.4397 - auc: 0.8844 - accuracy: 0.8323 - recall: 0.8323 - precision: 0.8323 - val_loss: 0.1670 - val_auc: 0.9966 - val_accuracy: 0.9690 - val_recall: 0.9690 - val_precision: 0.9690
Epoch 2/50
189/189 [==============================] - 58s 308ms/step - loss: 0.1304 - auc: 0.9965 - accuracy: 0.9756 - recall: 0.9756 - precision: 0.9756 - val_loss: 0.0993 - val_auc: 0.9982 - val_accuracy: 0.9761 - val_recall: 0.9761 - val_precision: 0.9761
Epoch 3/50
189/189 [==============================] - 61s 323ms/step - loss: 0.0942 - auc: 0.9972 - accuracy: 0.9790 - recall: 0.9790 - precision: 0.9790 - val_loss: 0.0792 - val_auc: 0.9983 - val_accuracy: 0.9787 - val_recall: 0.9787 - val_precision: 0.9787
Epoch 4/50
189/189 [==============================] - 59s 311ms/step - loss: 0.0794 - auc: 0.9975 - accuracy: 0.9795 - recall: 0.9795 - precision: 0.9795 - val_loss: 0.0696 - val_auc: 0.9984 - val_accuracy: 0.9802 - val_re

In [1]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_3_or_8 = (y_train1 == 3) | (y_train1 == 8)  
filter_3_or_8_test = (y_test1 == 3) | (y_test1 == 8)


x_train = x_train1[filter_3_or_8 ]  
y_train = y_train1[filter_3_or_8 ]  
  
x_test = x_test1[filter_3_or_8_test]  
y_test = y_test1[filter_3_or_8_test]  
  
  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 0
    else:
        y_train[i] = 1
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 0
    else:
        y_test[i] = 1

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=2,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=2,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=2

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
188/188 [==============================] - 84s 238ms/step - loss: 0.5944 - auc: 0.7586 - accuracy: 0.6938 - recall: 0.6938 - precision: 0.6938 - val_loss: 0.3945 - val_auc: 0.9486 - val_accuracy: 0.8972 - val_recall: 0.8972 - val_precision: 0.8972
Epoch 2/50
188/188 [==============================] - 24s 126ms/step - loss: 0.3233 - auc: 0.9613 - accuracy: 0.9135 - recall: 0.9135 - precision: 0.9135 - val_loss: 0.2681 - val_auc: 0.9708 - val_accuracy: 0.9335 - val_recall: 0.9335 - val_precision: 0.9335
Epoch 3/50
188/188 [==============================] - 24s 126ms/step - loss: 0.2393 - auc: 0.9744 - accuracy: 0.9327 - recall: 0.9327 - precision: 0.9327 - val_loss: 0.2139 - val_auc: 0.9798 - val_accuracy: 0.9395 - val_recall: 0.9395 - val_precision: 0.9395
Epoch 4/50
188/188 [==============================] - 24s 126ms/step - loss: 0.1992 - auc: 0.9802 - accuracy: 0.9412 - recall: 0.9412 - precision: 0.9412 - val_loss: 0.1826 - val_auc: 0.9840 - val_accuracy: 0.9466 - val_rec

In [2]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0
 
filter_1_3_6 = (y_train1 == 3) | (y_train1 == 6) | (y_train1 == 1) 
filter_1_3_6_test = (y_test1 == 3) | (y_test1 == 6) | (y_test1 == 1)

x_train = x_train1[filter_1_3_6 ]  
y_train = y_train1[filter_1_3_6 ]  
  
x_test = x_test1[filter_1_3_6_test]  
y_test = y_test1[filter_1_3_6_test]  
  
  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==1:
        y_train[i] = 0
    elif y_train[i]==3:
        y_train[i] = 1
    else:
        y_train[i] = 2
for i in range (len(y_test)):
    if y_test[i]==1:
        y_test[i] = 0
    elif y_test[i]==3:
        y_test[i] = 1
    else:
        y_test[i] = 2

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=3,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=3,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=3

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
294/294 [==============================] - 138s 301ms/step - loss: 0.5413 - auc: 0.9364 - accuracy: 0.8335 - recall: 0.7335 - precision: 0.8858 - val_loss: 0.2353 - val_auc: 0.9933 - val_accuracy: 0.9587 - val_recall: 0.9423 - val_precision: 0.9740
Epoch 2/50
294/294 [==============================] - 55s 189ms/step - loss: 0.1901 - auc: 0.9945 - accuracy: 0.9606 - recall: 0.9485 - precision: 0.9704 - val_loss: 0.1395 - val_auc: 0.9969 - val_accuracy: 0.9736 - val_recall: 0.9655 - val_precision: 0.9791
Epoch 3/50
294/294 [==============================] - 56s 189ms/step - loss: 0.1330 - auc: 0.9965 - accuracy: 0.9686 - recall: 0.9624 - precision: 0.9739 - val_loss: 0.1048 - val_auc: 0.9981 - val_accuracy: 0.9765 - val_recall: 0.9710 - val_precision: 0.9795
Epoch 4/50
294/294 [==============================] - 56s 191ms/step - loss: 0.1084 - auc: 0.9973 - accuracy: 0.9733 - recall: 0.9690 - precision: 0.9766 - val_loss: 0.0887 - val_auc: 0.9985 - val_accuracy: 0.9787 - val_re

In [3]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0
 
filter_01234 = (y_train1 == 0) | (y_train1 == 1) | (y_train1 == 2) | (y_train1 == 3) | (y_train1 == 4) 
filter_01234_test = (y_test1 == 0) | (y_test1 == 1) | (y_test1 == 2)  | (y_test1 == 3) | (y_test1 == 4)

x_train = x_train1[filter_01234 ]  
y_train = y_train1[filter_01234 ]  
  
x_test = x_test1[filter_01234_test]  
y_test = y_test1[filter_01234_test]  
  
  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)


batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=5,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=5,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=5

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    # loss='categorical_crossentropy',
    # optimizer='adam',
    # metrics=['AUC','accuracy','Recall','Precision'],
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    #metrics=[tf.keras.metrics.AUC(),tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall()],
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
479/479 [==============================] - 284s 419ms/step - loss: 0.7434 - auc: 0.9527 - accuracy: 0.8142 - recall: 0.5663 - precision: 0.9482 - val_loss: 0.3535 - val_auc: 0.9898 - val_accuracy: 0.9313 - val_recall: 0.8846 - val_precision: 0.9589
Epoch 2/50
479/479 [==============================] - 150s 312ms/step - loss: 0.2980 - auc: 0.9906 - accuracy: 0.9310 - recall: 0.8976 - precision: 0.9546 - val_loss: 0.2356 - val_auc: 0.9940 - val_accuracy: 0.9440 - val_recall: 0.9239 - val_precision: 0.9573
Epoch 3/50
479/479 [==============================] - 150s 314ms/step - loss: 0.2355 - auc: 0.9928 - accuracy: 0.9377 - recall: 0.9190 - precision: 0.9545 - val_loss: 0.1984 - val_auc: 0.9952 - val_accuracy: 0.9475 - val_recall: 0.9352 - val_precision: 0.9606
Epoch 4/50
479/479 [==============================] - 154s 321ms/step - loss: 0.2078 - auc: 0.9939 - accuracy: 0.9433 - recall: 0.9288 - precision: 0.9567 - val_loss: 0.1759 - val_auc: 0.9958 - val_accuracy: 0.9531 - val

In [4]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_01369 = (y_train1 == 0) | (y_train1 == 1) | (y_train1 == 3) | (y_train1 == 6) | (y_train1 == 9) 
filter_01369_test = (y_test1 == 0) | (y_test1 == 1) | (y_test1 == 3)  | (y_test1 == 6) | (y_test1 == 9)

x_train = x_train1[filter_01369]  
y_train = y_train1[filter_01369]  
  
x_test = x_test1[filter_01369_test]  
y_test = y_test1[filter_01369_test]  
  
  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 2
    elif y_train[i]==6:
        y_train[i] = 3
    elif y_train[i]==9:
        y_train[i] = 4
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 2
    elif y_test[i]==6:
        y_test[i] = 3
    elif y_test[i]==9:
        y_test[i] = 4


batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=5,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=5,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=5

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
480/480 [==============================] - 295s 440ms/step - loss: 0.7482 - auc: 0.9511 - accuracy: 0.8110 - recall: 0.5655 - precision: 0.9448 - val_loss: 0.3481 - val_auc: 0.9909 - val_accuracy: 0.9324 - val_recall: 0.8826 - val_precision: 0.9692
Epoch 2/50
480/480 [==============================] - 160s 333ms/step - loss: 0.2838 - auc: 0.9927 - accuracy: 0.9388 - recall: 0.9042 - precision: 0.9651 - val_loss: 0.2280 - val_auc: 0.9948 - val_accuracy: 0.9491 - val_recall: 0.9269 - val_precision: 0.9670
Epoch 3/50
480/480 [==============================] - 162s 337ms/step - loss: 0.2151 - auc: 0.9948 - accuracy: 0.9497 - recall: 0.9310 - precision: 0.9655 - val_loss: 0.1881 - val_auc: 0.9959 - val_accuracy: 0.9576 - val_recall: 0.9430 - val_precision: 0.9689
Epoch 4/50
480/480 [==============================] - 162s 337ms/step - loss: 0.1863 - auc: 0.9956 - accuracy: 0.9546 - recall: 0.9411 - precision: 0.9662 - val_loss: 0.1689 - val_auc: 0.9964 - val_accuracy: 0.9607 - val

In [5]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0
 
filter_036 = (y_train1 == 0) | (y_train1 == 3) | (y_train1 == 6) 
filter_036_test = (y_test1 == 0) | (y_test1 == 3)  | (y_test1 == 6) 

x_train = x_train1[filter_036]  
y_train = y_train1[filter_036]  
  
x_test = x_test1[filter_036_test]  
y_test = y_test1[filter_036_test]  

  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 1
    elif y_train[i]==6:
        y_train[i] = 2    
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 1
    elif y_test[i]==6:
        y_test[i] = 2
    


batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=3,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=3,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=3

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
281/281 [==============================] - 142s 332ms/step - loss: 0.4744 - auc: 0.9550 - accuracy: 0.8661 - recall: 0.8101 - precision: 0.9083 - val_loss: 0.2642 - val_auc: 0.9925 - val_accuracy: 0.9498 - val_recall: 0.9274 - val_precision: 0.9634
Epoch 2/50
281/281 [==============================] - 62s 222ms/step - loss: 0.2122 - auc: 0.9937 - accuracy: 0.9530 - recall: 0.9395 - precision: 0.9647 - val_loss: 0.1728 - val_auc: 0.9961 - val_accuracy: 0.9586 - val_recall: 0.9478 - val_precision: 0.9712
Epoch 3/50
281/281 [==============================] - 61s 219ms/step - loss: 0.1599 - auc: 0.9955 - accuracy: 0.9608 - recall: 0.9523 - precision: 0.9689 - val_loss: 0.1370 - val_auc: 0.9971 - val_accuracy: 0.9668 - val_recall: 0.9583 - val_precision: 0.9765
Epoch 4/50
281/281 [==============================] - 63s 226ms/step - loss: 0.1340 - auc: 0.9965 - accuracy: 0.9648 - recall: 0.9585 - precision: 0.9715 - val_loss: 0.1172 - val_auc: 0.9974 - val_accuracy: 0.9691 - val_re

In [8]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_1_or_5 = (y_train1 == 1) | (y_train1 == 5)  
filter_1_or_5_test = (y_test1 == 1) | (y_test1 == 5)


x_train = x_train1[filter_1_or_5]  
y_train = y_train1[filter_1_or_5]  
  
x_test = x_test1[filter_1_or_5_test]  
y_test = y_test1[filter_1_or_5_test]  
  

  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==5:
        y_train[i] = 0    
for i in range (len(y_test)):
    if y_test[i]==5:
        y_test[i] = 0
    

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=2,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=2,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=2

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
191/191 [==============================] - 110s 404ms/step - loss: 0.6105 - auc: 0.7417 - accuracy: 0.6916 - recall: 0.6916 - precision: 0.6916 - val_loss: 0.3348 - val_auc: 0.9711 - val_accuracy: 0.9250 - val_recall: 0.9250 - val_precision: 0.9250
Epoch 2/50
191/191 [==============================] - 52s 270ms/step - loss: 0.2447 - auc: 0.9843 - accuracy: 0.9429 - recall: 0.9429 - precision: 0.9429 - val_loss: 0.1631 - val_auc: 0.9931 - val_accuracy: 0.9694 - val_recall: 0.9694 - val_precision: 0.9694
Epoch 3/50
191/191 [==============================] - 46s 240ms/step - loss: 0.1416 - auc: 0.9938 - accuracy: 0.9676 - recall: 0.9676 - precision: 0.9676 - val_loss: 0.1062 - val_auc: 0.9968 - val_accuracy: 0.9803 - val_recall: 0.9803 - val_precision: 0.9803
Epoch 4/50
191/191 [==============================] - 45s 237ms/step - loss: 0.1025 - auc: 0.9961 - accuracy: 0.9742 - recall: 0.9742 - precision: 0.9742 - val_loss: 0.0803 - val_auc: 0.9979 - val_accuracy: 0.9827 - val_re

In [6]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_0369 = (y_train1 == 0) | (y_train1 == 3) | (y_train1 == 6) | (y_train1 == 9) 
filter_0369_test = (y_test1 == 0) |  (y_test1 == 3)  | (y_test1 == 6) | (y_test1 == 9)

x_train = x_train1[filter_0369]  
y_train = y_train1[filter_0369]  
  
x_test = x_test1[filter_0369_test]  
y_test = y_test1[filter_0369_test]  
  
  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 1
    elif y_train[i]==6:
        y_train[i] = 2
    elif y_train[i]==9:
        y_train[i] = 3
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 1
    elif y_test[i]==6:
        y_test[i] = 2
    elif y_test[i]==9:
        y_test[i] = 3


batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=4,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=4,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=4

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
374/374 [==============================] - 224s 427ms/step - loss: 0.7729 - auc: 0.9123 - accuracy: 0.7441 - recall: 0.5646 - precision: 0.9124 - val_loss: 0.3421 - val_auc: 0.9894 - val_accuracy: 0.9297 - val_recall: 0.8853 - val_precision: 0.9597
Epoch 2/50
374/374 [==============================] - 109s 290ms/step - loss: 0.2822 - auc: 0.9910 - accuracy: 0.9348 - recall: 0.9067 - precision: 0.9585 - val_loss: 0.2314 - val_auc: 0.9932 - val_accuracy: 0.9414 - val_recall: 0.9249 - val_precision: 0.9594
Epoch 3/50
374/374 [==============================] - 108s 288ms/step - loss: 0.2161 - auc: 0.9935 - accuracy: 0.9446 - recall: 0.9289 - precision: 0.9588 - val_loss: 0.1907 - val_auc: 0.9946 - val_accuracy: 0.9500 - val_recall: 0.9371 - val_precision: 0.9639
Epoch 4/50
374/374 [==============================] - 108s 288ms/step - loss: 0.1866 - auc: 0.9946 - accuracy: 0.9486 - recall: 0.9377 - precision: 0.9602 - val_loss: 0.1700 - val_auc: 0.9952 - val_accuracy: 0.9535 - val

In [7]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train ,x_test= x_train[..., np.newaxis] / 255.0,  x_test[..., np.newaxis] / 255.0
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=10,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=10,on_value=1,off_value=0)  

n = 10
blocks = 4
classes=10

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
938/938 [==============================] - 1375s 1s/step - loss: 0.9189 - auc: 0.9595 - accuracy: 0.7654 - recall: 0.5162 - precision: 0.9371 - val_loss: 0.5482 - val_auc: 0.9841 - val_accuracy: 0.8614 - val_recall: 0.7653 - val_precision: 0.9390
Epoch 2/50
938/938 [==============================] - 1166s 1s/step - loss: 0.5168 - auc: 0.9846 - accuracy: 0.8630 - recall: 0.7844 - precision: 0.9313 - val_loss: 0.4560 - val_auc: 0.9877 - val_accuracy: 0.8778 - val_recall: 0.8160 - val_precision: 0.9319
Epoch 3/50
938/938 [==============================] - 1176s 1s/step - loss: 0.4604 - auc: 0.9868 - accuracy: 0.8752 - recall: 0.8164 - precision: 0.9310 - val_loss: 0.4204 - val_auc: 0.9888 - val_accuracy: 0.8870 - val_recall: 0.8365 - val_precision: 0.9345
Epoch 4/50
938/938 [==============================] - 1159s 1s/step - loss: 0.4363 - auc: 0.9876 - accuracy: 0.8817 - recall: 0.8297 - precision: 0.9308 - val_loss: 0.4048 - val_auc: 0.9894 - val_accuracy: 0.8911 - val_recall:

In [9]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_3_or_9 = (y_train1 == 3) | (y_train1 == 9)  
filter_3_or_9_test = (y_test1 == 3) | (y_test1 == 9)


x_train = x_train1[filter_3_or_9]  
y_train = y_train1[filter_3_or_9]  
  
x_test = x_test1[filter_3_or_9_test]  
y_test = y_test1[filter_3_or_9_test]  
  

x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 0 
    else:
        y_train[i] = 1  
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 0
    else:
        y_test[i] = 1  
    

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=2,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=2,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=2

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
189/189 [==============================] - 118s 410ms/step - loss: 0.6020 - auc: 0.7492 - accuracy: 0.6917 - recall: 0.6917 - precision: 0.6917 - val_loss: 0.3083 - val_auc: 0.9546 - val_accuracy: 0.9044 - val_recall: 0.9044 - val_precision: 0.9044
Epoch 2/50
189/189 [==============================] - 48s 252ms/step - loss: 0.2498 - auc: 0.9707 - accuracy: 0.9228 - recall: 0.9228 - precision: 0.9228 - val_loss: 0.2071 - val_auc: 0.9776 - val_accuracy: 0.9326 - val_recall: 0.9326 - val_precision: 0.9326
Epoch 3/50
189/189 [==============================] - 50s 266ms/step - loss: 0.1861 - auc: 0.9825 - accuracy: 0.9391 - recall: 0.9391 - precision: 0.9391 - val_loss: 0.1689 - val_auc: 0.9841 - val_accuracy: 0.9435 - val_recall: 0.9435 - val_precision: 0.9435
Epoch 4/50
189/189 [==============================] - 53s 280ms/step - loss: 0.1560 - auc: 0.9870 - accuracy: 0.9508 - recall: 0.9508 - precision: 0.9508 - val_loss: 0.1480 - val_auc: 0.9872 - val_accuracy: 0.9534 - val_re

In [5]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc
K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_3_or_9 = (y_train1 == 3) | (y_train1 == 9)  
filter_3_or_9_test = (y_test1 == 3) | (y_test1 == 9)


x_train = x_train1[filter_3_or_9]  
y_train = y_train1[filter_3_or_9]  
  
x_test = x_test1[filter_3_or_9_test]  
y_test = y_test1[filter_3_or_9_test]  
  

x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 0 
    else:
        y_train[i] = 1  
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 0
    else:
        y_test[i] = 1  
    

batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=2,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=2,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=2

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.add(tf.keras.layers.Dropout(0.01))
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
189/189 [==============================] - 81s 253ms/step - loss: nan - auc: 0.8494 - accuracy: 0.7814 - recall: 0.7839 - precision: 0.7828 - val_loss: 0.2272 - val_auc: 0.9846 - val_accuracy: 0.9465 - val_recall: 0.9465 - val_precision: 0.9465
Epoch 2/50
189/189 [==============================] - 28s 146ms/step - loss: nan - auc: 0.9794 - accuracy: 0.9433 - recall: 0.9438 - precision: 0.9494 - val_loss: 0.1560 - val_auc: 0.9890 - val_accuracy: 0.9594 - val_recall: 0.9594 - val_precision: 0.9594
Epoch 3/50
189/189 [==============================] - 28s 146ms/step - loss: 0.3051 - auc: 0.9808 - accuracy: 0.9511 - recall: 0.9512 - precision: 0.9594 - val_loss: 0.1305 - val_auc: 0.9910 - val_accuracy: 0.9619 - val_recall: 0.9619 - val_precision: 0.9619
Epoch 4/50
189/189 [==============================] - 28s 146ms/step - loss: nan - auc: 0.9818 - accuracy: 0.9531 - recall: 0.9532 - precision: 0.9624 - val_loss: 0.1169 - val_auc: 0.9918 - val_accuracy: 0.9673 - val_recall: 0.96

In [6]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0
 
filter_036 = (y_train1 == 0) | (y_train1 == 3) | (y_train1 == 6) 
filter_036_test = (y_test1 == 0) | (y_test1 == 3)  | (y_test1 == 6) 

x_train = x_train1[filter_036]  
y_train = y_train1[filter_036]  
  
x_test = x_test1[filter_036_test]  
y_test = y_test1[filter_036_test]  

  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 1
    elif y_train[i]==6:
        y_train[i] = 2    
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 1
    elif y_test[i]==6:
        y_test[i] = 2
    


batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=3,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=3,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=3

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.add(tf.keras.layers.Dropout(0.01))
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
281/281 [==============================] - 138s 324ms/step - loss: 0.7181 - auc: 0.9142 - accuracy: 0.7915 - recall: 0.7139 - precision: 0.8539 - val_loss: 0.3148 - val_auc: 0.9868 - val_accuracy: 0.9332 - val_recall: 0.9081 - val_precision: 0.9547
Epoch 2/50
281/281 [==============================] - 60s 215ms/step - loss: 0.3848 - auc: 0.9832 - accuracy: 0.9383 - recall: 0.9215 - precision: 0.9607 - val_loss: 0.1863 - val_auc: 0.9953 - val_accuracy: 0.9596 - val_recall: 0.9474 - val_precision: 0.9695
Epoch 3/50
281/281 [==============================] - 60s 215ms/step - loss: 0.3292 - auc: 0.9863 - accuracy: 0.9491 - recall: 0.9397 - precision: 0.9678 - val_loss: 0.1468 - val_auc: 0.9964 - val_accuracy: 0.9613 - val_recall: 0.9535 - val_precision: 0.9706
Epoch 4/50
281/281 [==============================] - 60s 215ms/step - loss: 0.3007 - auc: 0.9878 - accuracy: 0.9546 - recall: 0.9469 - precision: 0.9704 - val_loss: 0.1286 - val_auc: 0.9968 - val_accuracy: 0.9640 - val_re

In [7]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc

K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0

filter_0369 = (y_train1 == 0) | (y_train1 == 3) | (y_train1 == 6) | (y_train1 == 9) 
filter_0369_test = (y_test1 == 0) |  (y_test1 == 3)  | (y_test1 == 6) | (y_test1 == 9)

x_train = x_train1[filter_0369]  
y_train = y_train1[filter_0369]  
  
x_test = x_test1[filter_0369_test]  
y_test = y_test1[filter_0369_test]  
  
  
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

for i in range (len(y_train)):
    if y_train[i]==3:
        y_train[i] = 1
    elif y_train[i]==6:
        y_train[i] = 2
    elif y_train[i]==9:
        y_train[i] = 3
for i in range (len(y_test)):
    if y_test[i]==3:
        y_test[i] = 1
    elif y_test[i]==6:
        y_test[i] = 2
    elif y_test[i]==9:
        y_test[i] = 3


batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)
y_train = tf.one_hot(y_train,depth=4,on_value=1,off_value=0)  
y_test = tf.one_hot(y_test,depth=4,on_value=1,off_value=0)  

n = 10
blocks = 3
classes=4

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    for k in range(classes):
        c = tc.Circuit(n, inputs=x)
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.add(tf.keras.layers.Dropout(0.01))
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(x_train_q, y_train, batch_size=64, epochs=50, validation_data=(x_test_q,y_test))

Epoch 1/50
374/374 [==============================] - 216s 405ms/step - loss: 0.7282 - auc: 0.9571 - accuracy: 0.8326 - recall: 0.6547 - precision: 0.9430 - val_loss: 0.3336 - val_auc: 0.9897 - val_accuracy: 0.9290 - val_recall: 0.8911 - val_precision: 0.9576
Epoch 2/50
374/374 [==============================] - 113s 301ms/step - loss: 0.4442 - auc: 0.9818 - accuracy: 0.9250 - recall: 0.8988 - precision: 0.9549 - val_loss: 0.2261 - val_auc: 0.9937 - val_accuracy: 0.9436 - val_recall: 0.9265 - val_precision: 0.9592
Epoch 3/50
374/374 [==============================] - 113s 302ms/step - loss: 0.3677 - auc: 0.9858 - accuracy: 0.9360 - recall: 0.9217 - precision: 0.9584 - val_loss: 0.1831 - val_auc: 0.9950 - val_accuracy: 0.9522 - val_recall: 0.9391 - val_precision: 0.9624
Epoch 4/50
374/374 [==============================] - 113s 302ms/step - loss: 0.3454 - auc: 0.9870 - accuracy: 0.9421 - recall: 0.9319 - precision: 0.9616 - val_loss: 0.1603 - val_auc: 0.9958 - val_accuracy: 0.9591 - val

In [12]:
import tensorflow as tf
from sklearn.decomposition import PCA 
import numpy as np
import tensorcircuit as tc
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.datasets as datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
K = tc.set_backend("tensorflow")


data_set, label  = datasets.load_iris(return_X_y = True) 
training_data, testing_data, training_label, testing_label = train_test_split(data_set, label, test_size=0.2, random_state=10)



# training_data = np.array(training_data)
training_label = np.array(training_label)
training_label = training_label.reshape(-1,)  
# testing_data = np.array(testing_data)
testing_label = np.array(testing_label)
testing_label = testing_label.reshape(-1,)  

y_train = tf.one_hot(training_label,depth=3,on_value=1,off_value=0)  
y_test = tf.one_hot(testing_label,depth=3,on_value=1,off_value=0)  


n = 4
blocks = 3
classes=3

def unitary(c,weights,wire1,wire2):
    c.ry(wire1,theta=weights[0])
    c.ry(wire2,theta=weights[1])
    c.cnot(wire1,wire2)

def qml(x, weights):
    output = []
    c = tc.Circuit(n)
    for k in range(classes):        
        for i in range(n):               
            c.rx(i,theta=x[i])     
        for j in range(blocks):
            for i in range(n-1):
                unitary(c,weights[k, j, i, :], i, i+1)                
        exp = K.real((K.sum(c.state())))
        output.append(exp)
    outputs = K.stack(output)
    outputs = K.reshape(outputs, [-1])
    return K.softmax(outputs)

qml_layer = tc.keras.QuantumLayer(qml, weights_shape=[classes,blocks, n-1, 2])
model = tf.keras.Sequential([qml_layer])
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['AUC','accuracy','Recall','Precision'],
)

model.fit(training_data, y_train, batch_size=16, epochs=50, validation_data=(testing_data,y_test))

Epoch 1/50
8/8 [==============================] - 84s 4s/step - loss: 1.0686 - auc: 0.6289 - accuracy: 0.4667 - recall: 0.1667 - precision: 0.3333 - val_loss: 1.0194 - val_auc: 0.6561 - val_accuracy: 0.5333 - val_recall: 0.1667 - val_precision: 0.3846
Epoch 2/50
8/8 [==============================] - 0s 15ms/step - loss: 0.9597 - auc: 0.7129 - accuracy: 0.5250 - recall: 0.2833 - precision: 0.5397 - val_loss: 0.9302 - val_auc: 0.7367 - val_accuracy: 0.5667 - val_recall: 0.4333 - val_precision: 0.8667
Epoch 3/50
8/8 [==============================] - 0s 13ms/step - loss: 0.8653 - auc: 0.7912 - accuracy: 0.6000 - recall: 0.4500 - precision: 0.7500 - val_loss: 0.8378 - val_auc: 0.8119 - val_accuracy: 0.6333 - val_recall: 0.5000 - val_precision: 0.9375
Epoch 4/50
8/8 [==============================] - 0s 13ms/step - loss: 0.7789 - auc: 0.8802 - accuracy: 0.7167 - recall: 0.5000 - precision: 0.8955 - val_loss: 0.7466 - val_auc: 0.9164 - val_accuracy: 0.8000 - val_recall: 0.5667 - val_precisi

In [11]:
import numpy as np
import tensorflow as tf
import tensorcircuit as tc
K = tc.set_backend("tensorflow")
(x_train1, y_train1), (x_test1, y_test1) = tf.keras.datasets.mnist.load_data()
x_train1,x_test1= x_train1[..., np.newaxis] / 255.0,  x_test1[..., np.newaxis] / 255.0
print("0 1 2 3 4 5 6 7 8 9")
print("sum:",len(x_train1)+len(x_test1))
print("ratio :",len(x_train1),"/",len(x_test1))

filter_1_or_5 = (y_train1 == 1) | (y_train1 == 5)  
filter_1_or_5_test = (y_test1 == 1) | (y_test1 == 5)

x_train = x_train1[filter_1_or_5 ]  
y_train = y_train1[filter_1_or_5 ]  
  
x_test = x_test1[filter_1_or_5_test]  
y_test = y_test1[filter_1_or_5_test]  

print("1 5")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))

filter_3_or_6 = (y_train1 == 3) | (y_train1 == 6)  
filter_3_or_6_test = (y_test1 == 3) | (y_test1 == 6)

x_train = x_train1[filter_3_or_6 ]  
y_train = y_train1[filter_3_or_6 ]  
  
x_test = x_test1[filter_3_or_6_test]  
y_test = y_test1[filter_3_or_6_test]  

print("3 6")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))

filter_3_or_8 = (y_train1 == 3) | (y_train1 == 8)  
filter_3_or_8_test = (y_test1 == 3) | (y_test1 == 8)

x_train = x_train1[filter_3_or_8 ]  
y_train = y_train1[filter_3_or_8 ]  
  
x_test = x_test1[filter_3_or_8_test]  
y_test = y_test1[filter_3_or_8_test]  

print("3 8")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))

filter_3_or_9 = (y_train1 == 3) | (y_train1 == 9)  
filter_3_or_9_test = (y_test1 == 3) | (y_test1 == 9)

x_train = x_train1[filter_3_or_9 ]  
y_train = y_train1[filter_3_or_9 ]  
  
x_test = x_test1[filter_3_or_9_test]  
y_test = y_test1[filter_3_or_9_test]  

print("3 9")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))



filter_036 = (y_train1 == 0) | (y_train1 == 3)  | (y_train1 == 6)  
filter_036_test = (y_test1 == 0) | (y_test1 == 3) | (y_test1 == 6)

x_train = x_train1[filter_036 ]  
y_train = y_train1[filter_036 ]  
  
x_test = x_test1[filter_036_test]  
y_test = y_test1[filter_036_test]  

print("0 3 6")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))


filter_136 = (y_train1 == 1) | (y_train1 == 3)  | (y_train1 == 6)  
filter_136_test = (y_test1 == 1) | (y_test1 == 3) | (y_test1 == 6)

x_train = x_train1[filter_136 ]  
y_train = y_train1[filter_136 ]  
  
x_test = x_test1[filter_136_test]  
y_test = y_test1[filter_136_test]  

print("1 3 6")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))


filter_0369 = (y_train1 == 0) | (y_train1 == 3)  | (y_train1 == 6)  | (y_train1 == 9)   
filter_0369_test = (y_test1 == 0) | (y_test1 == 3) | (y_test1 == 6) | (y_test1 == 9)

x_train = x_train1[filter_0369 ]  
y_train = y_train1[filter_0369 ]  
  
x_test = x_test1[filter_0369_test]  
y_test = y_test1[filter_0369_test]  

print("0 3 6 9")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))


filter_01369 = (y_train1 == 0) | (y_train1 == 1) |  (y_train1 == 3)  | (y_train1 == 6)  | (y_train1 == 9)   
filter_01369_test = (y_test1 == 0) | (y_test1 == 1) |(y_test1 == 3) | (y_test1 == 6) | (y_test1 == 9)

x_train = x_train1[filter_01369 ]  
y_train = y_train1[filter_01369 ]  
  
x_test = x_test1[filter_01369_test]  
y_test = y_test1[filter_01369_test]  

print("0 1 3 6 9")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))


filter_01234 = (y_train1 == 0) | (y_train1 == 1) |  (y_train1 == 2)  | (y_train1 == 3)  | (y_train1 == 4)   
filter_01234_test = (y_test1 == 0) | (y_test1 == 1) |(y_test1 == 2) | (y_test1 == 3) | (y_test1 == 4)

x_train = x_train1[filter_01234 ]  
y_train = y_train1[filter_01234]  
  
x_test = x_test1[filter_01234_test]  
y_test = y_test1[filter_01234_test]  

print("0 1 2 3 4")
print("sum:",len(x_train)+len(x_test))
print("ratio :",len(x_train),"/",len(x_test))

0 1 2 3 4 5 6 7 8 9
sum: 70000
ratio : 60000 / 10000
1 5
sum: 14190
ratio : 12163 / 2027
3 6
sum: 14017
ratio : 12049 / 1968
3 8
sum: 13966
ratio : 11982 / 1984
3 9
sum: 14099
ratio : 12080 / 2019
0 3 6
sum: 20920
ratio : 17972 / 2948
1 3 6
sum: 21894
ratio : 18791 / 3103
0 3 6 9
sum: 27878
ratio : 23921 / 3957
0 1 3 6 9
sum: 35755
ratio : 30663 / 5092
0 1 2 3 4
sum: 35735
ratio : 30596 / 5139
